In [1]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("recommender")\
    .getOrCreate()

sc = spark.sparkContext

22/12/04 21:46:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:

# load in the data
# data = sc.textFile("../ml-25m/ratings.csv.csv")
data = sc.textFile("ml-25m/ratings.csv")

In [4]:
# filter out header
header = data.first() #extract header
data = data.filter(lambda row: row != header)


In [5]:
# convert into a sequence of Rating objects
ratings = data.map(
  lambda l: l.split(',')
).map(
  lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))
)


In [6]:
# split into train and test
train, test = ratings.randomSplit([0.8, 0.2])



In [7]:
# train the model
K = 10
epochs = 10
model = ALS.train(train, K, epochs)


22/12/04 21:46:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 2): Attempting to kill Python Worker


22/12/04 21:46:17 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 3 (TID 3): Attempting to kill Python Worker


In [ ]:
# evaluate the model

# train
x = train.map(lambda p: (p[0], p[1]))
p = model.predictAll(x).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = train.map(lambda r: ((r[0], r[1]), r[2])).join(p)
# joins on first item: (user_id, movie_id)
# each row of result is: ((user_id, movie_id), (rating, prediction))
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("***** train mse: %s *****" % mse)


# test
x = test.map(lambda p: (p[0], p[1]))
p = model.predictAll(x).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = test.map(lambda r: ((r[0], r[1]), r[2])).join(p)
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("***** test mse: %s *****" % mse)

***** train mse: 0.5118851575641202 *****
